In [2]:
import pandas as pd
df = pd.read_csv('csv-data\\scraping_results.csv')
df.head()

,Locality,Zip Code,Type of Property,Subtype of Property,Price,Type of Sale,Number of Rooms,Livable Space (m2),Fully Equipped Kitchen,Furnished,...,Garden,Garden Area (m2),Swimming Pool,Surface of the Land (m2),Number of Facades,Construction Year,PEB,Primary Energy Consumption (kWh/m2),State of the Building,Url
0,Gent,9000,Apartment,apartment,229000.0,for-sale,1.0,44,0,0,...,0,NaN,0,NaN,2.0,1918.0,B,190.0,Good,https://www.immoweb.be/en/classified/apartment...
1,Uccle,1180,House,villa,1250000.0,for-sale,4.0,584,1,0,...,1,1085.0,0,1225.0,4.0,1932.0,G,402.0,To renovate,https://www.immoweb.be/en/classified/villa/for...
2,Etterbeek,1040,House,house,875000.0,for-sale,4.0,233,1,0,...,0,NaN,0,112.0,3.0,1929.0,G,563.0,To be done up,https://www.immoweb.be/en/classified/house/for...
3,Elversele,9140,House,villa,825000.0,for-sale,4.0,281,0,0,...,0,NaN,1,1450.0,4.0,2001.0,B,181.0,As new,https://www.immoweb.be/en/classified/villa/for...
4,Kalmthout,2920,House,manor-house,4750000.0,for-sale,3.0,326,0,0,...,0,NaN,0,34799.0,4.0,2015.0,A,95.0,As new,https://www.immoweb.be/en/classified/manor-hou...


In [3]:
df.shape

(10875, 23)

**How many rows and columns?**


Initially we had a dataset with 23 columns and 10875 inputs.

In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10875 entries, 0 to 10874
Data columns (total 23 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Locality                             10875 non-null  object 
 1   Zip Code                             10875 non-null  int64  
 2   Type of Property                     10875 non-null  object 
 3   Subtype of Property                  10875 non-null  object 
 4   Price                                10851 non-null  float64
 5   Type of Sale                         10875 non-null  object 
 6   Number of Rooms                      10744 non-null  float64
 7   Livable Space (m2)                   10875 non-null  int64  
 8   Fully Equipped Kitchen               10875 non-null  int64  
 9   Furnished                            10875 non-null  int64  
 10  Any Fireplace ?                      10875 non-null  int64  
 11  Terrace                     

We had some inputs that were missing price (24 cases) and number of rooms (131 cases), so we removed those cases as they would not be useful in further modeling.

In [5]:
df.dropna(subset=['Price'], inplace=True, ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10851 entries, 0 to 10850
Data columns (total 23 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Locality                             10851 non-null  object 
 1   Zip Code                             10851 non-null  int64  
 2   Type of Property                     10851 non-null  object 
 3   Subtype of Property                  10851 non-null  object 
 4   Price                                10851 non-null  float64
 5   Type of Sale                         10851 non-null  object 
 6   Number of Rooms                      10720 non-null  float64
 7   Livable Space (m2)                   10851 non-null  int64  
 8   Fully Equipped Kitchen               10851 non-null  int64  
 9   Furnished                            10851 non-null  int64  
 10  Any Fireplace ?                      10851 non-null  int64  
 11  Terrace                     

In [6]:
df.dropna(subset=['Number of Rooms'], inplace=True, ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10720 entries, 0 to 10719
Data columns (total 23 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Locality                             10720 non-null  object 
 1   Zip Code                             10720 non-null  int64  
 2   Type of Property                     10720 non-null  object 
 3   Subtype of Property                  10720 non-null  object 
 4   Price                                10720 non-null  float64
 5   Type of Sale                         10720 non-null  object 
 6   Number of Rooms                      10720 non-null  float64
 7   Livable Space (m2)                   10720 non-null  int64  
 8   Fully Equipped Kitchen               10720 non-null  int64  
 9   Furnished                            10720 non-null  int64  
 10  Any Fireplace ?                      10720 non-null  int64  
 11  Terrace                     

We had many missing values ​​for columns like Garden Area, Terrace Area and Surface of the land. So we filled these missing values ​​with 0 as the fact that these areas do not exist.

In [7]:
percentage_of_missing_garden_area = round(df['Garden Area (m2)'].isna().sum() / len(df) * 100, 2)
print(f"Percentage of missing garden area values: {percentage_of_missing_garden_area}%")
#print(f"Percentage of missing garden area values: {percentage_of_missing_garden_area:.2f}%")

percentage_of_missing_terrace_area = round(df['Terrace Area (m2)'].isna().sum() / len(df) * 100, 2)
print(f"Percentage of missing terrace area values: {percentage_of_missing_terrace_area}%")

percentage_of_missing_land_area = round(df['Surface of the Land (m2)'].isna().sum() / len(df) * 100, 2)
print(f"Percentage of missing values for surface of the land: {percentage_of_missing_land_area}%")

Percentage of missing garden area values: 69.31%
Percentage of missing terrace area values: 58.3%
Percentage of missing values for surface of the land: 34.33%


In [8]:
# Replace missing values in the 'Terrace Area (m2)' columns with 0
df['Terrace Area (m2)'] = df['Terrace Area (m2)'].fillna(0)

# Replace missing values in the 'Garden Area (m2)' columns with 0
df['Garden Area (m2)'] = df['Garden Area (m2)'].fillna(0)

# Replace missing values in the 'Surface of the Land (m2)' columns with 0
df['Surface of the Land (m2)'] = df['Surface of the Land (m2)'].fillna(0)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10720 entries, 0 to 10719
Data columns (total 23 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Locality                             10720 non-null  object 
 1   Zip Code                             10720 non-null  int64  
 2   Type of Property                     10720 non-null  object 
 3   Subtype of Property                  10720 non-null  object 
 4   Price                                10720 non-null  float64
 5   Type of Sale                         10720 non-null  object 
 6   Number of Rooms                      10720 non-null  float64
 7   Livable Space (m2)                   10720 non-null  int64  
 8   Fully Equipped Kitchen               10720 non-null  int64  
 9   Furnished                            10720 non-null  int64  
 10  Any Fireplace ?                      10720 non-null  int64  
 11  Terrace                     

We had 23.16% missing values ​​for the Number of Facades column. It was decided to modify this data according to the following rule:
1. Replace missing values in the 'Number of Facades' columns with 0
2. Replace 'Number of facades' with 1 where 'Type of Property' is apartment and 'Number of facades' is 0
3. Replace 'Number of Facades' with 2 where 'Subtype of Property' is 'duplex' or 'town-house' and 'Number of Facades' is 0
4. Replace 'Number of Facades' with 4 where 'Type of Property' is House and 'Number of Facades' is 0

In [10]:
percentage_of_missing_number_facades = round(df['Number of Facades'].isna().sum() / len(df) * 100, 2)
print(f"Percentage of missing values for number of facades: {percentage_of_missing_number_facades}%")

Percentage of missing values for number of facades: 23.16%


In [11]:
# Replace missing values in the 'Number of Facades' columns with 0
df['Number of Facades'] = df['Number of Facades'].fillna(0)

# Replace 'Number of facades' with 1 where 'Type of Property' is 0 (apartment) and 'Number of facades' is 0
df.loc[(df['Type of Property'] == 0) & (df['Number of Facades'] == 'Apartment'), 'Number of Facades'] = 1

# Replace 'Number of Facades' with 2 where 'Subtype of Property' is 'duplex' or 'town-house' and 'Number of Facades' is 0
df.loc[(df['Subtype of Property'].isin(['duplex', 'town-house'])) & (df['Number of Facades'] == 0), 'Number of Facades'] = 2

# Replace remaining 'Number of Facades' 0 values for remaining houses with 4
# Replace 'Number of Facades' with 4 where 'Type of Property' is 1 (House) and 'Number of Facades' is 0
df.loc[(df['Type of Property'] == 1) & (df['Number of Facades'] == 'House'), 'Number of Facades'] = 4

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10720 entries, 0 to 10719
Data columns (total 23 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Locality                             10720 non-null  object 
 1   Zip Code                             10720 non-null  int64  
 2   Type of Property                     10720 non-null  object 
 3   Subtype of Property                  10720 non-null  object 
 4   Price                                10720 non-null  float64
 5   Type of Sale                         10720 non-null  object 
 6   Number of Rooms                      10720 non-null  float64
 7   Livable Space (m2)                   10720 non-null  int64  
 8   Fully Equipped Kitchen               10720 non-null  int64  
 9   Furnished                            10720 non-null  int64  
 10  Any Fireplace ?                      10720 non-null  int64  
 11  Terrace                     

We had 5.74% and 12.13% missing values ​​for the PEB and State of the Building columns respectively, so we replaced these cases by indicating that the information was not specified.

In [13]:
percentage_of_missing_peb = round(df['PEB'].isna().sum() / len(df) * 100, 2)
print(f"Percentage of missing PEB values: {percentage_of_missing_peb}%")

percentage_of_missing_building_state = round(df['State of the Building'].isna().sum() / len(df) * 100, 2)
print(f"Percentage of missing values for state of the building: {percentage_of_missing_building_state}%")

Percentage of missing PEB values: 5.74%
Percentage of missing values for state of the building: 12.13%


In [14]:
df['PEB'] = df['PEB'].fillna('Not specified')
df['State of the Building'] = df['State of the Building'].fillna('Not specified')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10720 entries, 0 to 10719
Data columns (total 23 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Locality                             10720 non-null  object 
 1   Zip Code                             10720 non-null  int64  
 2   Type of Property                     10720 non-null  object 
 3   Subtype of Property                  10720 non-null  object 
 4   Price                                10720 non-null  float64
 5   Type of Sale                         10720 non-null  object 
 6   Number of Rooms                      10720 non-null  float64
 7   Livable Space (m2)                   10720 non-null  int64  
 8   Fully Equipped Kitchen               10720 non-null  int64  
 9   Furnished                            10720 non-null  int64  
 10  Any Fireplace ?                      10720 non-null  int64  
 11  Terrace                     

We had 10.62% missing values for the Energy consumption column. It was decided to replace them with the mean value of the entire range.

In [16]:
percentage_of_missing_energy_consumption = round(df['Primary Energy Consumption (kWh/m2)'].isna().sum() / len(df) * 100, 2)
print(f"Percentage of missing values for energy consumption: {percentage_of_missing_energy_consumption}%")

Percentage of missing values for energy consumption: 10.62%


In [17]:
# Replace missing values in the 'Primary Energy Consumption (kWh/m2)' columns with mean value
df['Primary Energy Consumption (kWh/m2)'] = df['Primary Energy Consumption (kWh/m2)'].fillna(round(df['Primary Energy Consumption (kWh/m2)'].mean(),0))

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10720 entries, 0 to 10719
Data columns (total 23 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Locality                             10720 non-null  object 
 1   Zip Code                             10720 non-null  int64  
 2   Type of Property                     10720 non-null  object 
 3   Subtype of Property                  10720 non-null  object 
 4   Price                                10720 non-null  float64
 5   Type of Sale                         10720 non-null  object 
 6   Number of Rooms                      10720 non-null  float64
 7   Livable Space (m2)                   10720 non-null  int64  
 8   Fully Equipped Kitchen               10720 non-null  int64  
 9   Furnished                            10720 non-null  int64  
 10  Any Fireplace ?                      10720 non-null  int64  
 11  Terrace                     

The Construction Year column was removed because it contained a lot of missing data that would be difficult to restore.


The "Sale Type" column had only one value, "for sale". Therefore, it was not of interest for the analysis and was also removed.

In [19]:
df = df.drop(columns=['Construction Year'])
df = df.drop(columns=['Type of Sale'])

In [20]:
# Find the index of the current 'Any Fireplace ?' column
fireplaces_index = df.columns.get_loc('Any Fireplace ?')

# Create the new 'Open Fire' column based on 'Any Fireplace ?'
df['Open Fire'] = df['Any Fireplace ?'].apply(lambda x: 0 if x == 0 else 1)

# Insert the new 'Open Fire' column at the same index
df.insert(fireplaces_index, 'Open Fire', df.pop('Open Fire'))
df = df.drop(columns=['Any Fireplace ?'])

In [21]:
df.columns

Index(['Locality', 'Zip Code', 'Type of Property', 'Subtype of Property',
       'Price', 'Number of Rooms', 'Livable Space (m2)',
       'Fully Equipped Kitchen', 'Furnished', 'Open Fire', 'Terrace',
       'Terrace Area (m2)', 'Garden', 'Garden Area (m2)', 'Swimming Pool',
       'Surface of the Land (m2)', 'Number of Facades', 'PEB',
       'Primary Energy Consumption (kWh/m2)', 'State of the Building', 'Url'],
      dtype='object')

For the Fully Equipped Kitchen column it was mentioned a value of 1 for Hyper-Equipped or Installed, and 0 for the rest.

It was noticed that some properties were presented on different web pages, so we removed these duplicates by comparing the values ​​for all columns except the URL column.

In [22]:
#Count duplicates for the same properties that have different URL.
df.duplicated(subset=df.columns.difference(['Url'])).value_counts()

False    10574
True       146
Name: count, dtype: int64

In [23]:
#Remove duplicates for the same properties that have different URL.
df.drop_duplicates(subset=df.columns.difference(['Url']), inplace=True)

In [24]:
df = df.drop(columns=['Url'])

In [25]:
df.shape

(10574, 20)

We added a Province column using a function based on the zip (postal) code values.

In [26]:
def add_province(zip):
    first_two_digits = int(str(zip)[:2])
    if 10 <= first_two_digits <= 12:
        return "Brussels-Capital Region"
    elif 13 <= first_two_digits <= 14:
        return "Province of Walloon Brabant"
    elif 15 <= first_two_digits <= 19 or 30 <= first_two_digits <= 34:
        return "Province of Flemish Brabant"
    elif 20 <= first_two_digits <= 29:
        return "Province of Antwerp"
    elif 35 <= first_two_digits <= 39:
        return "Province of Limburg"
    elif 40 <= first_two_digits <= 49:
        return "Province of Liège"
    elif 66 <= first_two_digits <= 69:
        return "Province of Luxembourg"
    elif 50 <= first_two_digits <= 56:
        return "Province of Namur"
    elif 60 <= first_two_digits <= 65 or 70 <= first_two_digits <= 79:
        return "Province of Hainaut"
    elif 80 <= first_two_digits <= 89:
        return "Province of West Flanders"
    elif 90 <= first_two_digits <= 99:
        return "Province of East Flanders"
    else:
        return "Unknown"

df['Province'] = df['Zip Code'].apply(add_province)

zip_code_index = df.columns.get_loc('Zip Code')  # Get the index of 'Zip Code'
df.insert(zip_code_index + 1, 'Province', df.pop('Province'))  # Insert 'Province' at the correct position

In [27]:
df.columns

Index(['Locality', 'Zip Code', 'Province', 'Type of Property',
       'Subtype of Property', 'Price', 'Number of Rooms', 'Livable Space (m2)',
       'Fully Equipped Kitchen', 'Furnished', 'Open Fire', 'Terrace',
       'Terrace Area (m2)', 'Garden', 'Garden Area (m2)', 'Swimming Pool',
       'Surface of the Land (m2)', 'Number of Facades', 'PEB',
       'Primary Energy Consumption (kWh/m2)', 'State of the Building'],
      dtype='object')

In [28]:
df.shape

(10574, 21)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10574 entries, 0 to 10719
Data columns (total 21 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Locality                             10574 non-null  object 
 1   Zip Code                             10574 non-null  int64  
 2   Province                             10574 non-null  object 
 3   Type of Property                     10574 non-null  object 
 4   Subtype of Property                  10574 non-null  object 
 5   Price                                10574 non-null  float64
 6   Number of Rooms                      10574 non-null  float64
 7   Livable Space (m2)                   10574 non-null  int64  
 8   Fully Equipped Kitchen               10574 non-null  int64  
 9   Furnished                            10574 non-null  int64  
 10  Open Fire                            10574 non-null  int64  
 11  Terrace                          

So we have:    

7 categorical columns ('Locality', 'Zip Code', 'Province', 'Type of Property', 'Subtype of Property', 'PEB', 'State of the Building'),    

8 numericals columns ('Price', 'Number of Rooms', 'Livable Space (m2)', 'Terrace Area (m2)', 'Garden Area (m2)', 'Surface of the Land (m2)', 'Number of Facades', 'Primary Energy Consumption (kWh/m2)'), 
  
6 boolean columns ('Fully Equipped Kitchen', 'Furnished', 'Open Fire', 'Terrace', 'Garden', 'Swimming Pool').

In [31]:
df.describe()

,Zip Code,Price,Number of Rooms,Livable Space (m2),Fully Equipped Kitchen,Furnished,Open Fire,Terrace,Terrace Area (m2),Garden,Garden Area (m2),Swimming Pool,Surface of the Land (m2),Number of Facades,Primary Energy Consumption (kWh/m2)
count,10574.000000,1.057400e+04,10574.000000,10574.000000,10574.000000,10574.000000,10574.000000,10574.000000,10574.000000,10574.000000,10574.000000,10574.000000,10574.000000,10574.000000,10574.000000
mean,4686.639020,7.431073e+05,3.634103,248.583980,0.604029,0.035181,0.090978,0.419047,15.155097,0.308303,332.594666,0.075279,1557.780405,2.303575,317.123983
std,3251.668304,8.476546e+05,2.152147,221.461482,0.489081,0.184245,0.287591,0.493426,62.605935,0.461814,1918.393334,0.263853,6472.426633,1.472582,1398.267250
min,1000.000000,2.750000e+04,1.000000,13.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,1640.000000,2.790000e+05,2.000000,111.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,166.000000
50%,3360.000000,5.199500e+05,3.000000,200.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,240.000000,2.000000,256.000000
75%,8400.000000,8.500000e+05,4.000000,311.000000,1.000000,0.000000,0.000000,1.000000,16.000000,1.000000,61.750000,0.000000,1000.000000,4.000000,334.000000
max,9992.000000,1.050000e+07,40.000000,5596.000000,1.000000,1.000000,1.000000,1.000000,3749.000000,1.000000,95000.000000,1.000000,160737.000000,9.000000,100000.000000


In [91]:
df.to_csv('csv-data\\cleaned_dataset_analysis.csv', index=False)